In [1]:
import pandas as pd
import os

In [2]:
from nltk.tokenize import word_tokenize

In [3]:
def tokenize_text(text):
    inside = False
    start = False
    text_lines_mod = []
    
    text_lines = word_tokenize(text)
    
    for line in text_lines:
        if "B__B-" in line:
            start = True
            inside = True
            token = line.split("B__B")[1].split("-")[1]
        elif "E__E-" in line:
            inside = False
        else:
            if not inside:
                line += " X X O"
            else:
                if start:
                    line += " X X B-" + token
                else:
                    line += " X X I-" + token
                start = False

            text_lines_mod.append(line)
            
    return text_lines_mod

In [4]:
def get_tokenized_text_lines(base_df, base_text_df):
    marked_texts = []

    for arquivo in base_df["file_id"].unique():
        entidades_df = base_df[base_df["file_id"] == arquivo].copy().reset_index(drop=True)[["val_ent", "ner_token", "loc_begin", "loc_end", "file_id"]]
        text = base_text_df[base_text_df["file_id"] == arquivo]["raw_text"].iloc[0]

        entidades_df = entidades_df.sort_values(["loc_end"], ascending=False).reset_index(drop=True)

        for token, begin, end in entidades_df[["ner_token", "loc_begin", "loc_end"]].itertuples(index=False):
            mark_token_begin = " B__B-" + token + "-B__B "
            mark_token_end = " E__E-" + token + "-E__E "

            text = text[:end] + mark_token_end +  text[end:]
            text = text[:begin] + mark_token_begin +  text[begin:]

        marked_texts.append(text)
        
    marked_texts_df = pd.DataFrame(pd.Series(marked_texts), columns=["marked_text"])
    marked_texts_df["file_id"] = base_df["file_id"].unique()
    
    #return marked_texts_df

    marked_texts_df["treated_text"] = clean_text_dodfs(marked_texts_df["marked_text"])
    marked_texts_df["treated_text"] = marked_texts_df.apply(lambda row: clean_text_by_word(row["treated_text"]), axis=1)
    
    marked_texts_df["tokenized_text"] = marked_texts_df.apply(lambda row: tokenize_text(row["treated_text"]), axis=1)
    
    return marked_texts_df

In [11]:
def clean_text_dodfs(dodfs_texts):

    start_page_patterns = [r"\nPÁGINA\s([0-9]{1,5})", r"\nDIÁRIO\sOFICIAL\sDO\sDISTRITO\sFEDERAL",
                           r"\nNº(.+?)2([0-9]{3})", r"\nxx([a-z]{0,10}) Diário Oficial do Distrito Federal xx([a-z]{0,10})",
                           r"\nDiário Oficial do Distrito Federal"]

    end_page_patterns = [r"Documento assinado digitalmente conforme MP nº 2.200-2 de 24/08/2001, que institui a",
                            r"Infraestrutura de Chaves Públicas Brasileira ICP-Brasil",
                            r"Este documento pode ser verificado no endereço eletrônico",
                            r"http://wwwin.gov.br/autenticidade.html",
                            r"pelo código ([0-9]{15,18})",
                            r"\nDocumento assinado digitalmente, original em https://www.dodf.df.gov.br",
                        r"http:/Awwwin.gov.br/autenticidade.html",
                        r"Documento assinado digitalmente conforme MP nº 2.200-2 de 24/08/2001,",
                        r"\nque institui a\n",
                        r"\nhttp://www.in.gov.br/autenticidade.html",
                        r"\nhttp://www.in.gov.brautenticidade html",
                        r"Documento assinado digitalmente conforme MP n 2.200-2 de 24/08/2001, que institui a .",
                        r"http://www.in.gov.brautenticidade html,"]

    middle_page_patterns = [r"xx([a-z]{1,10}) ", r" xx([a-z]{1,10})", r"xx([a-z]{1,10})"]
    
    special_middle_page_patterns = [r"\n-\n", r"\n- -\n", r"\n- - -\n", r"\n[\.\,\-\—]\n", r"— -", r". -",
                                   r"\r[\.\,\-\—]\r", r"\n-\r", r"\r"]
    
    start_page_patterns = "|".join(start_page_patterns)
    middle_page_patterns = "|".join(middle_page_patterns)
    end_page_patterns = "|".join(end_page_patterns)
    
    page_patterns = [start_page_patterns, middle_page_patterns, end_page_patterns]
    page_patterns = "|".join(page_patterns)
    
    return dodfs_texts.str.replace(page_patterns, "", regex=True).replace(special_middle_page_patterns, "\n", regex=True)

In [13]:
def clean_text_by_word(text):
    a = "\n".join([l for l in text.split("\n") if l != ""])
    words = a.replace("\n", " ").split(" ")
    words = [w for w in words if w != ""]
    
    m_words = []
    dash_cut = False

    for i in range(len(words)):
        word = words[i]

        if (word[-1] == "-") and (i+1)<len(words):
            word = word[:-1] + words[i+1]
            i += 1

        m_words.append(word)
        
    return " ".join(m_words)

In [5]:
def specific_parse(base_df, func=None):
    if func == None:
        return base_df
    else:
        return func(base_df)

In [6]:
# Definindo os tokens com base no título de entidade de cada tipo de ato

ner_tokens_extrato_contrato = {
    "Número do ajuste": "NUM_AJUSTE",
    "Órgão contratante": "CONTRATANTE",
    "Entidade contratada": "CONTRATADA",
    "Entidades convenentes": "CONVENENTE",
    "Processo do GDF": "PROCESSO",
    "Objeto do ajuste": "OBJ_AJUSTE",
    "Data de assinatura do ajuste": "DATA_ASSINATURA",
    "Vigência do ajuste": "VIGENCIA",
    "Valor do ajuste": "VALOR",
    "Código da unidade orçamentária": "CODIGO_UO",
    "Programa de trabalho": "PT",
    "Natureza da despesa": "ND",
    "Nota de empenho": "NE",
    "Órgão/entidade licitante": "ORGAO_LICITANTE",
    "Número da licitação que gerou o contrato": "NUM_LICITACAO",
    "Órgão gerenciador da ata": "OG_ATA",
    "Identificação de dispensa": "IDENT_DISPENSA",
    "Fundamento legal da dispensa ou inexigibilidade": "FUND_DISPENSA"
}

ner_tokens_aditamento_contratual = {
    "Órgão contratante": "CONTRATANTE",
    "Número do contrato": "NUM_CONTRATO",
    "Número do termo aditivo": "NUM_ADITIVO",
    "Objeto do aditamento contratual": "OBJ_ADITIVO"
}

ner_tokens_aviso_licitacao = {
    "Órgão/entidade licitante": "ORGAO_LICITANTE",
    "Número da licitação": "NUM_LICITACAO",
    "Objeto da licitação": "OBJ_LICITACAO",
    "Modalidade de licitação": "MODALIDADE_LICITACAO",
    "Processo do GDF": "PROCESSO",
    "Valor estimado da contratação": "VALOR_ESTIMADO",
    "Data de abertura da licitação": "DATA_ABERTURA",
    "Sistema de compras utilizado": "SISTEMA_COMPRAS",
    "Código da licitação no sistema de compras utilizado": "CODIGO_SISTEMA_COMPRAS",
    "Tipo de objeto": "TIPO_OBJ"
}

ner_tokens_aviso_revogacao = {
    "Órgão/entidade licitante": "ORGAO_LICITANTE",
    "Número da licitação": "NUM_LICITACAO",
    "Modalidade de licitação": "MODALIDADE_LICITACAO",
    "Identificação revogação/anulação": "IDENT_REVOGACAO_ANULACAO"
}

ner_tokens_aviso_suspensao_licitacao = {
    "Órgão/entidade licitante": "ORGAO_LICITANTE",
    "Número da licitação": "NUM_LICITACAO",
    "Modalidade de licitação": "MODALIDADE_LICITACAO",
    "Prazo da suspensão": "PRAZO_SUSPENSAO",
    "Decisão do TCDF que determinou a suspensão": "DECISAO_TCDF"
}

In [7]:
atos_data_df = pd.read_parquet("reviewed_entities.parquet")
atos_data_text_df = pd.read_parquet("reviewed_acts.parquet")

In [8]:
# Inserir função de tratamento de dados em specific_parse caso haja (filtragem de tipos de ato, por exemplo)
filt_atos_data_df = specific_parse(atos_data_df)

In [9]:
atos_data_df["ner_token"] = ""

atos_data_df.loc[atos_data_df["act_type"].isin(["extrato_de_contrato_ou_convenio"]), "ner_token"] = atos_data_df.loc[atos_data_df["act_type"].isin(["extrato_de_contrato_ou_convenio"]), "title_ent"].replace(ner_tokens_extrato_contrato)
atos_data_df.loc[atos_data_df["act_type"].isin(["aviso_de_aditamento_contratual"]), "ner_token"] = atos_data_df.loc[atos_data_df["act_type"].isin(["aviso_de_aditamento_contratual"]), "title_ent"].replace(ner_tokens_aditamento_contratual)
atos_data_df.loc[atos_data_df["act_type"].isin(["aviso_de_licitacao"]), "ner_token"] = atos_data_df.loc[atos_data_df["act_type"].isin(["aviso_de_licitacao"]), "title_ent"].replace(ner_tokens_aviso_licitacao)
atos_data_df.loc[atos_data_df["act_type"].isin(["aviso_de_revogacao_anulacao_de_licitacao"]), "ner_token"] = atos_data_df.loc[atos_data_df["act_type"].isin(["aviso_de_revogacao_anulacao_de_licitacao"]), "title_ent"].replace(ner_tokens_aviso_revogacao)
atos_data_df.loc[atos_data_df["act_type"].isin(["aviso_de_suspensao_de_licitacao"]), "ner_token"] = atos_data_df.loc[atos_data_df["act_type"].isin(["aviso_de_suspensao_de_licitacao"]), "title_ent"].replace(ner_tokens_aviso_suspensao_licitacao)

In [14]:
full_tokenized_texts = get_tokenized_text_lines(filt_atos_data_df, atos_data_text_df)

In [15]:
full_tokenized_texts["treated_text"] = full_tokenized_texts["tokenized_text"].apply(lambda row: " ".join([w.split()[0] for w in row]))